# 요약


---

GPT 3.5 사용으로 약 40% ~ 44% 의 정확도를 보였다. 낮은 정확도의 이유는 GPT 3.5 가 긴 문맥을 제대로 이해하지 못하기 때문으로 보인다. 또한 Tokenizer 의 길이 한도 때문에 문맥을 온전히 전달하지 못했다.

# 해결 과정


---


## 1. 문제에 대한 개념 주입 필요
해당 문제는 비교적 간단하게 해결되었다. 인터넷 상에 있는 모의고사 pdf 파일을 JSON 화 하는 함수를 선언하였다. 이를 통해 GPT 에게 예시 문제와 정답을 알려주어, GPT가 수능 문제에 대한 이해를 할 수 있게끔 했다.

## 2. GPT 3.5 의 문해력 부족
3.5 버전의 경우 4버전에 비해 문해력이 부족하다. 이로 인해 긴 문맥을 이해하기 어렵다. 이에 대한 해법으로 Cross Attention 을 도입했다. JSON 으로 변환한 모의고사를 이용해 Cross Attention 레이어를 학습시켰다. 해당 레이어는 질문에 따라 지문에서 더 주목해야 하는 단어를 선정한다. 중요할 것으로 보이는 단어에는 앞뒤에 * 를 붙여 GPT 에게 전달한다. GPT에게 *가 붙은 단어는 중요하다는 사실 역시 프롬프팅한다. 이를 통해 GPT 3.5 에 추가적인 Transformer 레이어를 도입한 것과 비슷한 효과를 거두기를 기대했다.

## 3. Tokenizer 길이 제한
가장 해결하기 어려운 부분이었다. 512라는 토큰 제한으로 인해 지문 전체를 gpt에게 전달할 수 없었다. 이 때문에 Chunking 을 도입했다. 지문을 여러 부분으로 쪼개고, GPT 에게 한 Chunk 씩 전달할 때마다 앞선 내용과 함께 요약하도록 했다. 요약을 위한 Chunk 들 역시 Cross Attention 을 통한 *강조 표시가 있어, 문맥을 효과적으로 요약하기를 기대했다. 하지만 결과는 좋지 않았다. 오히려 Chunking 을 도입하기 전보다 정확도가 5%가량 하락했다. 이는 GPT 3.5 의 요약 성능이 좋지 못해, 많은 정보가 손실되었기 때문으로 보인다.

결과적으로 해당 페이지의 코드에는 Chunking 이 적용되어있지 않다. 적용된 코드는 아래에서 확인 가능하다.

(문맥 Chunking 코드 : https://github.com/raccoonD0G/AIPlus99/blob/main/Homework3/Homework3_1_1.py)



In [ ]:

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import pandas as pd
from openai import OpenAI
# from google.colab import userdata
import re
import fitz  # PyMuPDF

import fitz
import re

from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    true = labels.argmax(axis=-1)  # one-hot to index
    acc = accuracy_score(true, preds)
    return {"accuracy": acc}

def extract_questions_from_pdf(pdf_path, max_questions=34):
    doc = fitz.open(pdf_path)
    full_text = "\n".join([page.get_text() for page in doc])

    range_tags = re.findall(r'\[(\d+)[~～](\d+)\]', full_text)
    paragraph_ranges = [(int(start), int(end)) for start, end in range_tags]

    raw_blocks = re.split(r'\n?\s*(\d{1,2})[.)번]?\s+', full_text)

    questions = []
    current_shared_paragraph = ""
    current_range = None
    prev_text = raw_blocks[0]

    for i in range(1, len(raw_blocks) - 1, 2):
        number = int(raw_blocks[i])
        if number > max_questions:
            break  # 34번까지만

        body = raw_blocks[i + 1]
        in_shared_range = False
        for r in paragraph_ranges:
            if r[0] <= number <= r[1]:
                in_shared_range = True
                if current_range != r:
                    pattern = re.compile(rf'\[{r[0]}[~～]{r[1]}\](.*?)(?=\n\s*{r[0]}[.)번])', re.DOTALL)
                    match = pattern.search(full_text)
                    current_shared_paragraph = match.group(1).strip() if match else ""
                    current_range = r
                break

        choices_match = re.findall(
            r'①(.*?)②(.*?)③(.*?)④(.*?)⑤(.*?)(?=\n\d{1,2}[.)번]|\n\n|\Z)',
            body,
            re.DOTALL
        )
        if choices_match:
            choices = choices_match[0]
            question_match = re.split(r'①', body, maxsplit=1)
            question_text = question_match[0].strip()
            paragraph = current_shared_paragraph if in_shared_range else prev_text.strip()

            questions.append({
                "number": number,
                "paragraph": paragraph,
                "question": question_text,
                "choices": [o.strip() for o in choices]
            })

            prev_text = ""
        else:
            prev_text += "\n" + body

    return questions


def extract_common_subject_answers(pdf_path, max_questions=34):
    doc = fitz.open(pdf_path)
    text = doc[0].get_text()

    symbol_to_index = {'①': 0, '②': 1, '③': 2, '④': 3, '⑤': 4}
    matches = re.findall(r'(\d+)\s*([①-⑤])\s*\d+', text)

    answers = {}
    for number, symbol in matches:
        num = int(number)
        if 1 <= num <= max_questions:
            answers[num] = symbol_to_index[symbol]

    correct_indices = [answers[i] for i in range(1, max_questions + 1) if i in answers]

    if len(correct_indices) < max_questions:
        print(f"경고: 추출된 정답 수 {len(correct_indices)}개가 기대값 {max_questions}보다 적습니다.")

    return correct_indices

questions = extract_questions_from_pdf("Exam2.pdf")
answers = extract_common_subject_answers("Answer2.pdf")

import json

def print_questions(questions, max_items=None):
    if max_items is not None:
        questions = questions[:max_items]

    print(json.dumps(questions, ensure_ascii=False, indent=2))


# 정답 붙이기
for i in range(min(len(questions), len(answers))):
    questions[i]["answer"] = answers[i]

print (answers);
# 출력
print_questions(questions, max_items=34)


import torch
import torch.nn as nn
from transformers import AutoTokenizer
from transformers import AutoModel

class ExamCrossAttention(nn.Module):
    def __init__(self, num_labels=5, hidden_size=768, num_heads=8):
        super().__init__()
        self.bert = AutoModel.from_pretrained("klue/bert-base")

        for param in self.bert.parameters():
            param.requires_grad = False

        self.cross_attention = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=num_heads, batch_first=True)
        self.layer_norm = nn.LayerNorm(hidden_size)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.LayerNorm(512),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),

            nn.Linear(256, 128),
            nn.LeakyReLU(0.1),

            nn.Linear(128, num_labels)
        )

        self.last_attn_weights = None

    def forward(self, input_ids_question, attention_mask_question, input_ids_paragraph, attention_mask_paragraph, labels=None):
        # 1. 질문 인코딩
        q_output = self.bert(input_ids=input_ids_question, attention_mask=attention_mask_question)
        question_embed = q_output.last_hidden_state  # (B, Lq, D)

        # 2. 지문 인코딩
        c_output = self.bert(input_ids=input_ids_paragraph, attention_mask=attention_mask_paragraph)
        paragraph_embed = c_output.last_hidden_state  # (B, Lc, D)

        # 3. Cross-Attention
        attn_output, attn_weights = self.cross_attention(
            query=question_embed,                # (B, Lq, D)
            key=paragraph_embed,                   # (B, Lc, D)
            value=paragraph_embed,                 # (B, Lc, D)
            key_padding_mask=(attention_mask_paragraph == 0),  # False = attend, True = mask
            average_attn_weights=False,
        )

        self.last_attn_weights = attn_weights.detach()

        # 4. Residual + LayerNorm
        attended = self.layer_norm(question_embed + attn_output)  # (B, Lq, D)

        # 5. 질문 토큰에 대한 pooling
        mean_pooled = attended.mean(dim=1)       # (B, D)
        max_pooled = attended.max(dim=1).values  # (B, D)

        pooled = (mean_pooled + max_pooled) / 2  # (B, D)


        logits = self.classifier(pooled)

        if labels is not None:
            labels_idx = labels.argmax(dim=1)
            loss = nn.CrossEntropyLoss()(logits, labels_idx)
            return {"loss": loss, "logits": logits}

        return {"logits": logits}



from torch.utils.data import Dataset

class MCQDataset(Dataset):
    def __init__(self, questions, tokenizer, max_q_len=512, max_p_len=512):
        self.questions = [q for q in questions if "answer" in q]
        self.tokenizer = tokenizer
        self.max_q_len = max_q_len
        self.max_p_len = max_p_len

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        q = self.questions[idx]
        question_text = q["question"]
        paragraph_text = q["paragraph"]
        label_idx = q["answer"]

        # 질문 인코딩
        q_encoding = self.tokenizer(
            question_text,
            padding="max_length",
            truncation=True,
            max_length=self.max_q_len,
            return_tensors="pt"
        )

        # 지문 인코딩
        p_encoding = self.tokenizer(
            paragraph_text,
            padding="max_length",
            truncation=True,
            max_length=self.max_p_len,
            return_tensors="pt"
        )

        # 레이블 (one-hot)
        one_hot = torch.zeros(5)
        one_hot[label_idx] = 1

        return {
            "input_ids_question": q_encoding["input_ids"].squeeze(0),
            "attention_mask_question": q_encoding["attention_mask"].squeeze(0),
            "input_ids_paragraph": p_encoding["input_ids"].squeeze(0),
            "attention_mask_paragraph": p_encoding["attention_mask"].squeeze(0),
            "labels": one_hot
        }


from transformers import TrainingArguments, Trainer

train_questions = []
test_questions = []

for i in range(1, 2):
    exam_file = f"Exam{i}.pdf"
    answer_file = f"Answer{i}.pdf"

    # 문제 및 정답 추출
    pdf_questions = extract_questions_from_pdf(exam_file)
    pdf_answers = extract_common_subject_answers(answer_file)

    for j in range(min(len(pdf_questions), len(pdf_answers))):
        pdf_questions[j]["answer"] = pdf_answers[j]

    train_questions.extend(pdf_questions)
    test_questions.extend(pdf_questions)



print(f"훈련용 문제 수: {len(train_questions)}")
print(f"테스트용 문제 수: {len(test_questions)}")

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
tokenizer.model_max_length = 512

for i in range(1, 2):
    exam_file = f"Exam{i}.pdf"
    answer_file = f"Answer{i}.pdf"

    questions = extract_questions_from_pdf(exam_file)
    answers = extract_common_subject_answers(answer_file)

    print(f"[Exam{i}] 문제 수: {len(questions)}, 정답 수: {len(answers)}")

# 학습용 Dataset 생성
train_dataset = MCQDataset(train_questions, tokenizer)

# 테스트용 Dataset 생성
test_dataset = MCQDataset(test_questions, tokenizer)

model = ExamCrossAttention()


training_args = TrainingArguments(
    output_dir="./Exam",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=1
)


from transformers import TrainerCallback
from tqdm import tqdm

class LogAccuracyCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        torch.save(model.state_dict(), "trained_model.pt")


trainer  = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[LogAccuracyCallback()]
)

import matplotlib.pyplot as plt

def plot_accuracy_from_trainer(trainer):
    log_history = trainer.state.log_history
    eval_steps = []
    eval_accs = []

    for entry in log_history:
        if "eval_accuracy" in entry:
            step = entry.get("epoch", entry.get("step", None))
            eval_steps.append(step)
            eval_accs.append(entry["eval_accuracy"])

    if not eval_accs:
        print("정확도 기록이 없습니다.")
        return

    plt.plot(eval_steps, eval_accs, marker='o', label="Eval Accuracy")
    plt.title("Evaluation Accuracy per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

trainer.train()

model.load_state_dict(torch.load("trained_model.pt"))

from openai import OpenAI

client = OpenAI(api_key="")

def format_example(q):
    paragraph, _ = highlight_words_with_cross_attention(q['question'], q['paragraph'], model, tokenizer)
    return f"""[지문] :
{paragraph}

[질문] :
{q['question']}

[선택지] :
1) {q['choices'][0]}
2) {q['choices'][1]}
3) {q['choices'][2]}
4) {q['choices'][3]}
5) {q['choices'][4]}

[정답] :
{q['answer'] + 1}"""


url = "https://raw.githubusercontent.com/NomaDamas/KICE_slayer_AI_Korean/master/data/2023_11_KICE.json"
df = pd.read_json(url)

# 미리보기
df.head()


MAX_TOTAL_TOKENS = 8000

def count_tokens(text):
    return len(tokenizer.encode(text))

import torch
from transformers import DistilBertModel, DistilBertTokenizer

import string

def highlight_words_with_cross_attention(question_text, paragraph_text, model, tokenizer, max_q_len=512, max_p_len=512, top_k=20, device="cuda"):

    # 1. Tokenize
    q_enc = tokenizer(question_text, return_tensors="pt", padding="max_length", truncation=True, max_length=max_q_len)
    p_enc = tokenizer(paragraph_text, return_tensors="pt", padding="max_length", truncation=True, max_length=max_p_len)

    input_ids_q = q_enc["input_ids"].to(device)
    mask_q = q_enc["attention_mask"].to(device)
    input_ids_p = p_enc["input_ids"].to(device)
    mask_p = p_enc["attention_mask"].to(device)

    # 2. Cross Attention Forward
    with torch.no_grad():
        model.eval()
        _ = model(
            input_ids_question=input_ids_q,
            attention_mask_question=mask_q,
            input_ids_paragraph=input_ids_p,
            attention_mask_paragraph=mask_p
        )
    attn_weights = model.last_attn_weights  # (B, H, Lq, Lp)

    # 3. 평균 attention
    avg_attn = attn_weights.mean(dim=1)[0]     # (Lq, Lp)
    token_scores = avg_attn.mean(dim=0)        # (Lp,)

    # 4. 토큰을 단어 단위로 병합하며 각 단어에 스코어 매핑
    tokens = tokenizer.convert_ids_to_tokens(input_ids_p[0], skip_special_tokens=True)

    words = []
    scores = []
    current_word = ""
    current_score_total = 0
    current_token_count = 0

    for token, score in zip(tokens, token_scores):
        if token.startswith("##"):
            token = token[2:]
            current_word += token
        else:
            if current_word:
                words.append(current_word)
                avg_score = current_score_total / max(current_token_count, 1)
                scores.append(avg_score)
            current_word = token
            current_score_total = 0
            current_token_count = 0
        current_score_total += score.item()
        current_token_count += 1

    if current_word:
        words.append(current_word)
        scores.append(current_score_total / max(current_token_count, 1))

    # top-k 단어 강조
    top_k = min(top_k, len(scores))
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]

    highlighted_words = []
    for i, word in enumerate(words):
        # 문장 부호인지 확인
        if word in string.punctuation or word in ["``", "''", "”", "“", "’", "‘", "(", ")", ",", ".", "!", "?", "·", "…", ":", ";"]:
            # 문장 부호는 바로 앞 단어에 붙이기
            if highlighted_words:
                highlighted_words[-1] += word
            else:
                highlighted_words.append(word)
        else:
            if i in top_indices:
                highlighted_words.append(f"*{word}*")
            else:
                highlighted_words.append(word)

    # 문장 구성
    highlighted_paragraph = " ".join(highlighted_words)
    return highlighted_paragraph, scores



def ask_gpt_with_few_shot(few_shot_pool, problem):
    q_text = problem["question"]
    if "question_plus" in problem:
        q_text += "\n\n" + problem["question_plus"]

    paragraph = highlight_words_with_cross_attention(problem['question'], problem['paragraph'], model, tokenizer)[0]
    choices = problem["choices"]

    print(f"[지문 하이라이트]\n{paragraph}")

    choice_text = "\n".join([f"{i+1}) {opt}" for i, opt in enumerate(choices)])

    static_problem_block = f"""
---

이제 새로운 문제입니다:

[지문] :
{paragraph}

[질문] :
{q_text}

[선택지] :
{choice_text}

[정답] :
"""

    system_prompt = """
다음은 한국 수능형 객관식 문제입니다.
아래 [지문]과 [질문]을 읽고, [질문]에 가장 잘 부합하는 [선택지] 중 하나를 골라주세요
[정답]은 1, 2, 3, 4, 5 중 하나입니다.
[정답]을 숫자 하나로만 출력하세요.
그 외 텍스트를 포함하지 마세요.

글에 중요한 단어는 *강조*되어 있습니다.

아래는 예시입니다:
"""

    # 예시 수를 줄여가며 프롬프트 구성
    for n in range(3, -1, -1):
        few_shots = random.sample(few_shot_pool, n) if n > 0 else []
        few_shot_prompt = "\n\n---\n\n".join([format_example(q) for q in few_shots])
        full_prompt = f"{system_prompt}\n\n{few_shot_prompt}\n{static_problem_block}"

        if count_tokens(full_prompt) <= MAX_TOTAL_TOKENS:
            break
    else:
        print("예시 0개로도 토큰 수 초과. 건너뜀")
        return -1

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": full_prompt}],
            temperature=0,
            max_tokens=5,
        )
        print(f"최종 전송본 : {full_prompt}")
        content = response.choices[0].message.content.strip()
        match = re.search(r"\b([1-5])\b", content)
        print(f"GPT 답변 : {content}")
        print(f"GPT 답변 정리 : {match}")
        return int(match.group()) if match else -1
    except Exception as e:
        print(f"오류 발생: {e}")
        return -1


import random

# 전체 평가 루프
def evaluate_dataframe_with_few_shot(df, train_questions):
    correct = 0
    total_score = 0
    total_questions = 0

    train_questions = [q for q in train_questions if "answer" in q]

    for index, row in tqdm(df.iterrows(), total=len(df), desc="Few-shot GPT 채점 중"):
        paragraph = row["paragraph"]
        problems = row["problems"]

        for i, problem in enumerate(problems):
            # 질문 내용 구성
            problem["paragraph"] = paragraph
            q_text = problem["question"]
            if "question_plus" in problem:
                q_text += "\n\n" + problem["question_plus"]

            # 예시 3개 가져오기
            few_shots = random.sample(train_questions, 3)
            if len(few_shots) < 3:
                continue  # 예시 부족하면 skip

            correct_answer = problem["answer"]
            score = problem.get("score", 1)

            print(f"\n[문제 {total_questions + 1}]")
            print(f"[질문] {q_text}")
            print(f"[정답] {correct_answer}")

            gpt_answer = ask_gpt_with_few_shot(few_shots, problem)

            print(f"[GPT 선택] {gpt_answer}")

            if gpt_answer not in range(1, 6):
                print(f"GPT 정답 오류 : {gpt_answer}")

            if gpt_answer == correct_answer:
                correct += 1
                total_score += score
            total_questions += 1

    accuracy = correct / total_questions * 100 if total_questions > 0 else 0
    return accuracy, total_score, total_questions

# 채점 실행
accuracy, total_score, total_questions = evaluate_dataframe_with_few_shot(df, train_questions)

print(f"\n총 문제 수: {total_questions}")
print(f"정답 수: {int(accuracy * total_questions / 100)}")
print(f"정답률: {accuracy:.2f}%")
print(f"총 점수: {total_score}점")



[0, 3, 0, 2, 0, 4, 4, 2, 0, 1, 3, 1, 3, 1, 4, 4, 0, 3, 1, 2, 3, 0, 3, 4, 4, 3, 2, 4, 3, 2, 2, 3, 2, 1]
[
  {
    "number": 1,
    "paragraph": "다음 글을 읽고 물음에 답하시오.\n글을 읽는 동안 독자의 사고 과정을 밝힐 수 있는 방법 중 \n하나가 눈동자 움직임 분석 방법이다. 이것은 사고 과정이 \n눈동자의 움직임에 반영된다고 보고 그 특성을 분석하는 방법\n이다.\n눈동자 움직임에 주목한 연구에 따르면, 글을 읽을 때 \n독자는 자신이 중요하다고 판단한 단어나 생소하다고 생각한 \n단어를 중심으로 읽는다. 글을 읽을 때 독자는 눈동자를 \n단어에 멈추는 고정, 고정과 고정 사이에 일어나는 도약을 \n보였는데, 도약은 한 단어에서 다음 단어로 이동하는 짧은 \n도약과 단어를 건너뛰는 긴 도약으로 구분된다. 고정이 \n관찰될 때는 단어의 의미 이해가 이루어졌지만, 도약이 \n관찰될 때는 건너뛴 단어의 의미 이해가 이루어지지 않았다. \n글을 읽을 때 독자가 생각하는 단어의 중요도나 친숙함에 \n따라 눈동자의 고정 시간과 횟수, 도약의 길이와 방향도 \n달랐다. 독자가 중요하거나 생소하다고 생각한 단어일수록 \n고정 시간이 길었다. 이러한 단어는 독자가 글의 진행 \n방향대로 읽어 가다가 되돌아와 다시 읽는 경우도 있어 \n고정 횟수도 많았고, 이때의 도약은 글의 진행 방향과는 \n다르게 나타났다. 중요한 단어나 생소한 단어가 연속될 \n때는 그 단어마다 눈동자가 멈추면서 도약의 길이가 짧았다.\n눈동자 움직임의 양상은 독자의 읽기 능력이 발달하면서 변화\n한다. 읽기 능력이 발달하면 이전과 같은 수준의 글을 읽거나 \n전에 읽었던 글을 다시 읽을 때, 단어마다 눈동자를 고정하\n지는 않게 되어 ㉠이전보다 고정 횟수와 고정 시간이 줄어들고 \n단어를 건너뛰는 긴 도약이 자주 일어나는 모습이 관찰된다. \n학습 경험과 독서 경험이 쌓이면서 글의 구조에

훈련용 문제 수: 34
테스트용 문제 수: 34
[Exam1] 문제 수: 34, 정답 수: 34


Epoch,Training Loss,Validation Loss,Accuracy
1,1.573200,1.590794,0.235294
2,1.607700,1.574433,0.264706
3,1.573000,1.561935,0.264706
4,1.630100,1.550227,0.264706
5,1.593200,1.540445,0.264706
6,1.599700,1.530489,0.264706
7,1.561700,1.520032,0.264706
8,1.551400,1.510427,0.294118
9,1.540200,1.499583,0.294118
10,1.584400,1.487597,0.294118


Few-shot GPT 채점 중:   0%|          | 0/11 [00:00<?, ?it/s]


[문제 1]
[질문] 윗글의 내용과 일치하지 않는 것은?
[정답] 4
[지문 하이라이트]
사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. *독서의* 즐거움에는 여러 가지가 있겠지만 그 중심에는‘ 소통의 즐거움’ 이 있다. *독자는* 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. *직접* 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 *세계를바라볼* 수 있다. *이때* 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 *요인*, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 *소통* 과정에서 다른 의미를 구성할 수 있다.[ A]( 이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. *독자는* 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. *전자의* 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. *또한* 후자의 경우 책에는 *없는* 독자의 경험에서 답을 찾을 수 있다. *이런* 질문들을 풍부히 *생성하고* 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.) 한편 독자는( *다른* 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. *비슷한* 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 *소통* 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 *양상이* 더 다양해지고 있다. *자신의* 독서 경험을 담은 글이나 

...

Few-shot GPT 채점 중:  91%|█████████ | 10/11 [00:48<00:03,  3.96s/it]

최종 전송본 : 
다음은 한국 수능형 객관식 문제입니다.
아래 [지문]과 [질문]을 읽고, [질문]에 가장 잘 부합하는 [선택지] 중 하나를 골라주세요
[정답]은 1, 2, 3, 4, 5 중 하나입니다.
[정답]을 숫자 하나로만 출력하세요.
그 외 텍스트를 포함하지 마세요.

글에 중요한 단어는 *강조*되어 있습니다.

아래는 예시입니다:


[지문] : 
다음 글을 읽고 물음에 답하시오.( 가) 향아 너의 *고운* 얼굴 조석으로 우물가에 비최이던 오래지 않은 옛날로 가자 수수럭거리는 수수밭 사이 걸찍스런 웃음들 들려 나오며 호미와 바구니를 든 환한 얼굴 그림처럼 나타나던 석양…… 구슬처럼 흘러가는 냇물가 *맨발을* 담그고 늘어앉아 빨래들을 두드리던 전설같은 풍속으로 돌아가자 눈동자를 보아라 향아 회올리는 무지갯빛 허울의 눈부심에 넋 빼앗기지 말고 철따라 푸짐히 두레를 먹던 *마을로* 돌아가자 미끈덩한 기생충의 생리와 허식에 인이 *배기기* *전으로* 눈빛 아침처럼 빛나던 우리들의 고향 병들지 않은 젊음으로 *찾아* 가자꾸나 이 문제지에 관한 저작권은 한국교육과정평가원에 있습니다. 12 *12* 20 향아 허물어질까 두렵노라 얼굴 생김새 맞지 않는 발돋움의 *흉낼랑* *그만* *내자* 들국화처럼 소박한 목숨을 가꾸기 위하여 맨발을 벗고 콩바심 하던 차라리 그 미개지에로 가자 달이 뜨는 명절밤 비단치마를 나부끼며 떼지어 춤추던 전설같은 풍속으로 돌아가자 냇물 굽이치는 싱싱한 마음밭으로 돌아가자.- 신동엽, *｢* 향아 *｣*-( 나) 이사온 *그는* 이상한 사람이었다 그의 *집* 담장들은 모두 빛나는 유리들로 세워졌다 골목에서 놀고 있는 부주의한 *아이들이* 잠깐의 실수 때문에 풍성한 햇빛을 복사해내는 그 유리 담장을 박살내곤 했다 그러나 얘들아, 상관없다 유리는 또 갈아 끼우면 되지 마음껏 이 골목에서 놀렴 유리를 깬 아이는 얼굴이 새빨개졌지만 이상한 표정을 짓던 다른 아이들은 아이들답게 곧 즐거워했다 견고한 송판으로 담을 쌓으면 어떨까 주장하는 아이는, *그

최종 전송본 : 
다음은 한국 수능형 객관식 문제입니다.
아래 [지문]과 [질문]을 읽고, [질문]에 가장 잘 부합하는 [선택지] 중 하나를 골라주세요
[정답]은 1, 2, 3, 4, 5 중 하나입니다.
[정답]을 숫자 하나로만 출력하세요.
그 외 텍스트를 포함하지 마세요.

글에 중요한 단어는 *강조*되어 있습니다.

아래는 예시입니다:


[지문] : 
다음 글을 읽고 물음에 답하시오.( 가) 향아 너의 고운 얼굴 조석으로 우물가에 *비최이던* 오래지 않은 옛날로 가자 수수럭거리는 수수밭 사이 걸찍스런 웃음들 *들려* 나오며 호미와 바구니를 든 환한 얼굴 그림처럼 나타나던 석양…… 구슬처럼 흘러가는 냇물가 *맨발을* 담그고 늘어앉아 빨래들을 두드리던 전설같은 풍속으로 돌아가자 눈동자를 보아라 향아 *회올리는* 무지갯빛 허울의 눈부심에 넋 빼앗기지 말고 철따라 푸짐히 두레를 먹던 마을로 돌아가자 미끈덩한 기생충의 생리와 *허식에* 인이 배기기 전으로 눈빛 *아침처럼* 빛나던 우리들의 고향 병들지 않은 젊음으로 *찾아* 가자꾸나 *이* 문제지에 관한 저작권은 한국교육과정평가원에 있습니다. 12 *12* 20 향아 허물어질까 두렵노라 얼굴 생김새 맞지 않는 발돋움의 *흉낼랑* *그만* *내자* 들국화처럼 소박한 목숨을 가꾸기 위하여 맨발을 벗고 콩바심 *하던* 차라리 그 미개지에로 가자 달이 뜨는 명절밤 비단치마를 나부끼며 떼지어 춤추던 전설같은 풍속으로 돌아가자 냇물 굽이치는 싱싱한 마음밭으로 돌아가자.- 신동엽, *｢* 향아 *｣*-( 나) 이사온 그는 이상한 사람이었다 그의 집 담장들은 모두 빛나는 유리들로 세워졌다 골목에서 놀고 있는 부주의한 아이들이 잠깐의 실수 때문에 풍성한 햇빛을 복사해내는 그 유리 담장을 박살내곤 했다 그러나 얘들아, 상관없다 유리는 또 갈아 *끼우면* 되지 마음껏 이 골목에서 놀렴 유리를 깬 아이는 얼굴이 새빨개졌지만 이상한 표정을 짓던 다른 아이들은 아이들답게 곧 즐거워했다 견고한 송판으로 담을 쌓으면 어떨까 주장하는 아이는, 

Few-shot GPT 채점 중: 100%|██████████| 11/11 [00:51<00:00,  4.64s/it]

최종 전송본 : 
다음은 한국 수능형 객관식 문제입니다.
아래 [지문]과 [질문]을 읽고, [질문]에 가장 잘 부합하는 [선택지] 중 하나를 골라주세요
[정답]은 1, 2, 3, 4, 5 중 하나입니다.
[정답]을 숫자 하나로만 출력하세요.
그 외 텍스트를 포함하지 마세요.

글에 중요한 단어는 *강조*되어 있습니다.

아래는 예시입니다:


[지문] : 
다음 글을 읽고 물음에 답하시오. 글을 읽으려면 글자 읽기, 요약, 추론 등의 읽기 *기능*, 어휘력, 읽기 흥미나 동기 등이 필요하다. *글* 읽는 능력이 발달하려면 읽기에 필요한 이러한 요소를 잘 갖추어야 한다. *읽기* 요소들 중 어휘력 발달에 관한 연구들에서는, 학년이 올라감에 따라 높은 학생들과 낮은 학생들 간의 어휘력 격차가 점점 더 커짐이 보고되었다. 여기서 *어휘력* 격차는 읽기의 양과 관련된다. 즉 *어휘력이* 높으면 이를 바탕으로 점점 더 많이 읽게 되고, 많이 읽을 수록 글 속의 어휘를 습득할 기회가 많아지며, 이것이 다시 어휘력을 높인다는 것이다. 반대로, 어휘력이 부족하면 읽는 양도 적어지고 어휘 습득의 기회도 줄어 다시 어휘력이 상대적 으로 부족하게 됨으로써, 나중에는 커져 버린 격차를 극복 하는 데에 많은 노력이 필요하게 된다. 이렇게 *읽기* 요소를 잘 갖춘 독자는 점점 더 잘 읽게 되어 그렇지 않은 독자와의 차이가 갈수록 *커지게* 되는데, 이를 매튜 효과로 설명하기도 한다. 매튜 *효과란* 사회적 명성이나 물질적 자산이 많을수록 그로 인해 더 많이 가지게 되고, 그 결과 그렇지 않은 사람과의 차이가 점점 *커지는* 현상을 일컫는다. 이는 *주로* 사회학에서 사용되었으나 읽기에도 적용된다. 그러나 *읽는* 능력을 매튜 효과로만 설명하는 데에는 문제가 있다. *우선*, *읽기와* 관련된 요소들에서 매튜 효과가 항상 나타나는 것은 아니다. 인지나 *정서의* 발달은 개인마다 다르며, 한 개인 안에서도 그 속도는 시기마다 다르기 때문이다. 예컨대 읽기 *흥미나* 동기의 경우, 어릴 때